In [1]:
import os
import sys
sys.path.append('..')

In [2]:
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning
import pytorchvideo.data
import pytorchvideo.models.resnet
import pytorchvideo.models.slowfast
import torch.utils.data
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.model_selection import KFold
from torch.utils.data import Subset
from sklearn.metrics import roc_auc_score
from fvcore.common.config import CfgNode
from IPython.display import clear_output

In [3]:
from constants import kinetics_data_path, cloud_data_path
import optimizer
from dataset import my_video_dataset, my_video_dataset_from_dataframe
from defaults import get_cfg
from transforms import get_kinetics_train_transform, get_kinetics_val_transform
from utils import get_metrics
cfg = get_cfg()

In [4]:
def make_kinetics_resnet():
#   return pytorchvideo.models.slowfast.create_slowfast(
#       input_channels=(3,3), # RGB input from Kinetics
#       model_depth=50, # For the tutorial let's just use a 50 layer network
#       model_num_class=2, # Kinetics has 400 classes so we need out final head to align
#   )
  return pytorchvideo.models.resnet.create_resnet(
      input_channel=3, # RGB input from Kinetics
      model_depth=50, # For the tutorial let's just use a 50 layer network
      model_num_class=2, # Kinetics has 400 classes so we need out final head to align
      norm=nn.BatchNorm3d,
      activation=nn.ReLU,
      
  )

In [5]:
def make_resnet_feature_extractor():
    model = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=True)

    for param in model.parameters():
        param.requires_grad = False

    model.blocks[-1].proj = nn.Linear(2048, 2)

    return model

In [6]:
def make_slowfast_feature_extractor():
    model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)

    for param in model.parameters():
        param.requires_grad = False

    model.blocks[-1].proj = nn.Linear(2304, 2)

    return model

In [7]:
class VideoClassificationLightningModule(pytorch_lightning.LightningModule):
  def __init__(self, optim_cfg={}):
      super().__init__()
      self.model = make_slowfast_feature_extractor()
      self.optim_cfg = optim_cfg

  def forward(self, x):
      return self.model(x)

  def training_step(self, batch, batch_idx):

      # learning rate scheduling
      epoch_exact = self.current_epoch + float(batch_idx) / self.trainer.num_training_batches
      self.log("epoch_exact", epoch_exact)
      lr = optimizer.get_epoch_lr(epoch_exact, self.optim_cfg)
      
      optimizer.set_lr(self.optimizers().optimizer, lr)
      self.log("learning_rate", lr)

      # The model expects a video tensor of shape (B, C, T, H, W), which is the
      # format provided by the dataset
      y_hat = self.model(batch["video"])

      # Compute cross entropy loss, loss.backwards will be called behind the scenes
      # by PyTorchLightning after being returned from this method.
      loss = F.cross_entropy(y_hat, batch["label"])

      # Log the train loss to Tensorboard
      self.log("train_loss", loss.item())

      return loss

  def validation_step(self, batch, batch_idx):
      y_hat = self.model(batch["video"])
      loss = F.cross_entropy(y_hat, batch["label"])
      self.log("val_loss", loss)
      return (y_hat[:,1], batch["label"])

  def validation_epoch_end(self, validation_step_outputs):
        all_outputs = torch.cat([o[0] for o in validation_step_outputs]).cpu()
        all_labels = torch.cat([o[1] for o in validation_step_outputs]).cpu()

        try:
            val_auc = roc_auc_score(all_labels, all_outputs)
            self.log('val_auc', val_auc)
        except ValueError:
            pass

  def configure_optimizers(self):
      """
      Setup the Adam optimizer. Note, that this function also can return a lr scheduler, which is
      usually useful for training video models.
      """
      return optimizer.get_optimizer(self.model, self.optim_cfg)
      

In [8]:
def train_from_scratch():
    cfg = get_cfg()
    cfg.merge_from_other_cfg(CfgNode({
        'SOLVER': {
            'OPTIMIZING_METHOD': 'sgd',
            'BASE_LR': 0.1,
            'LR_POLICY': 'cosine',
            'MOMENTUM': 0.9,
            'WEIGHT_DECAY': 1e-4,
            'WARMUP_EPOCHS': 0.0,
            'WARMUP_START_LR': 0.01
        }
    }))
    classification_module = VideoClassificationLightningModule(optim_cfg=cfg)
    data_module = KineticsDataModule()
    trainer = pytorch_lightning.Trainer(
        callbacks=[EarlyStopping(monitor="val_loss", mode="min")],
        accelerator='gpu',
        log_every_n_steps=1,
        max_epochs=50
    )
    trainer.fit(classification_module, data_module)

In [9]:
def do_fold(train_ds, test_ds):
    # data loaders
    data_loader_train = torch.utils.data.DataLoader(
        train_ds, batch_size=8, shuffle=True, num_workers=10,
        collate_fn=None)
    data_loader_val = torch.utils.data.DataLoader(
        test_ds, batch_size=8, shuffle=False, num_workers=10,
        collate_fn=None)

    cfg = get_cfg()
    cfg.merge_from_other_cfg(CfgNode({
        'SOLVER': {
            'OPTIMIZING_METHOD': 'sgd',
            'BASE_LR': 0.1,
            'LR_POLICY': 'none',
            'MOMENTUM': 0.9,
            'WEIGHT_DECAY': 1e-4,
        }
    }))
    
    system = VideoClassificationLightningModule(optim_cfg=cfg)
    trainer = pytorch_lightning.Trainer(
        callbacks=[EarlyStopping(monitor="val_loss", mode="min")],
        accelerator='gpu',
        log_every_n_steps=1,
        max_epochs=50)
    trainer.fit(system, data_loader_train, data_loader_val)

    trainer.test(system, data_loader_val)
    return system.test_results

In [10]:
def do_cross_validation(dataset, metrics_name='binary'):
    seed = 22
    cv_splits = KFold(n_splits=10, random_state=seed, shuffle=True).split(range(len(dataset)))

    outputs = torch.empty((len(dataset),))
    for f, (train_idx, test_idx) in enumerate(cv_splits):
        # create datasets    
        train_ds = Subset(dataset, train_idx)
        test_ds = Subset(dataset, test_idx)

        fold_outputs = do_fold(train_ds, test_ds)
        outputs[test_idx] = fold_outputs['proba'].cpu()
        clear_output(wait=True)

    labels = torch.Tensor(dataset.get_all_labels())
    run_metrics = get_metrics(outputs, labels, metrics_name)

    return outputs, run_metrics

In [11]:
examples = pd.read_csv('../dataset/computational_examples.csv')
examples = examples[examples['condition'] == 'video']
examples['filename'] = examples['hash']+'.mp4'
video_path = os.path.join(cloud_data_path, 'laughter_data', 'ml_datasets', 'tight', 'video')

In [12]:
examples.head()

,Unnamed: 0.1,Unnamed: 0,person,cam,hit_id,condition,calibration,hash,ini_time,end_time,...,gt_laughter,is_laughter,confidence,intensity,attempt,pressed_key,onset,offset,rating_hash,filename
0,0,0,25,1,9c45e4f0c5442e796eb93e73e94dc6c2dfca7b9c4c54ff...,video,False,1170917790b51bc5a8dacacc4d8ed8c410b7ea6bb7ea4b...,7360.29,7361.54,...,True,True,7,4,0,True,3.336670,6.639973,7af591213b827db95c12c56e76e0b1fe518f2088d11aad...,1170917790b51bc5a8dacacc4d8ed8c410b7ea6bb7ea4b...
1,1947,1947,25,1,bff9b86d833a595e6fe5a54f45093fa168cda45db1143e...,video,False,1170917790b51bc5a8dacacc4d8ed8c410b7ea6bb7ea4b...,7360.29,7361.54,...,True,True,1,6,0,True,2.569236,6.639973,25df21dc0f25e11a7c4aba77e502269d42a7bb548044f2...,1170917790b51bc5a8dacacc4d8ed8c410b7ea6bb7ea4b...
6,979,979,35,3,f4c9842cec7be99eeaaea36d0c7d077c4d5d94596dc731...,video,False,11bc9d8aca57ab2aef4c5305b080fa49c08665d9e94190...,2216.02,2216.54,...,True,False,3,4,0,False,NaN,NaN,e56c0edb6a0870ea94b570507d246cb56c9ab3b1919a05...,11bc9d8aca57ab2aef4c5305b080fa49c08665d9e94190...
7,2744,2744,35,3,eecc0cf5d634ce45a98cbbda30c922f2a2cfcb1877124c...,video,False,11bc9d8aca57ab2aef4c5305b080fa49c08665d9e94190...,2216.02,2216.54,...,True,False,5,4,0,False,NaN,NaN,4c6a7bdcbec4e912d140f9a9ae4a196e0b55f0e5eff6b3...,11bc9d8aca57ab2aef4c5305b080fa49c08665d9e94190...
12,2,2,1,4,9c45e4f0c5442e796eb93e73e94dc6c2dfca7b9c4c54ff...,video,False,c1d181e74dbdbce1e51d7d0bfd6e036913896dd1f22856...,3346.30,3347.70,...,True,True,2,2,0,True,2.736069,3.203203,07d4ee10402ea059d0a3791fd35fbaab20149aeb6ffb99...,c1d181e74dbdbce1e51d7d0bfd6e036913896dd1f22856...


In [13]:
dataset = my_video_dataset_from_dataframe(
    examples_df=examples,
    video_path=video_path,
    video_path_prefix=video_path,
    clip_sampler=pytorchvideo.data.make_clip_sampler("random", 2),
    transform=get_kinetics_train_transform(32, 256, True),
    decode_audio=False,
    file_path_key='filename',
    label_key='pressed_key'
)

In [14]:
do_cross_validation(dataset, metrics_name='binary')

Using cache found in /home/jose/.cache/torch/hub/facebookresearch_pytorchvideo_main
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | Net  | 33.6 M
-------------------------------
4.6 K     Trainable params
33.6 M    Non-trainable params
33.6 M    Total params
134.596   Total estimated model params size (MB)


bn 0, non bn 2, zero 0 no grad 330


Sanity Checking: 0it [00:00, ?it/s]

/home/jose/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

MisconfigurationException: No `test_step()` method defined to run `Trainer.test`.